In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(df,hue='Outcome')

In [ ]:
x=df.drop('Outcome',axis=1).values

y=df['Outcome'].values

# Train test split

In [ ]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=2)

**Importing necessary libraries**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Creating tensors

In [ ]:
xtrain=torch.FloatTensor(xtrain)
xtest=torch.FloatTensor(xtest)
ytrain=torch.LongTensor(ytrain)
ytest=torch.LongTensor(ytest)

In [ ]:
df.shape

# Creating ANN model

In [ ]:
class ANN_model(nn.Module):
    def __init__(self,input_features=8,hidden_1=20,hidden_2=20,out_fea=2):
        super().__init__()
        self.conn1=nn.Linear(input_features,hidden_1)
        self.conn2=nn.Linear(hidden_1,hidden_2)
        self.out=nn.Linear(hidden_2,out_fea)
        
    def forward(self,x):
        x=F.relu(self.conn1(x))
        x=F.relu(self.conn2(x))
        x=self.out(x)
        return x

**Instantiate the model**

In [ ]:
torch.manual_seed(20)

model=ANN_model()

In [ ]:
model.parameters

**It gives the summary of the model**

# Backward propogation

**Loss and optimization function**

In [ ]:
loss_function=nn.CrossEntropyLoss()

optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

In [ ]:
final_loss=[]

for i in range(500):
    i=i+1
    ypred=model.forward(xtrain)
    loss=loss_function(ypred,ytrain)
    final_loss.append(loss)
    if i%10==0:
        print('Epoch number {} and the loss is {}'.format(i,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Plotting the loss function

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(500),final_loss)
plt.xlabel('Epochs')
plt.ylabel('Loss')

**The loss is gradually decreased from the first epoch**

In [ ]:
for i,dat in enumerate(xtest[0]):
    print(i,dat)

In [ ]:
prediction=[]
with torch.no_grad():
    for i,dat in enumerate(xtest):
        ypred=model(dat)
        prediction.append(ypred.argmax().item())
        print(ypred.argmax().item())

# Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

conf=confusion_matrix(ytest,prediction)

sns.heatmap(conf)

**True positives and false negatives are predicting nicely than the rest**

# Accuracy

In [ ]:
from sklearn.metrics import accuracy_score

acc=accuracy_score(ytest,prediction)

acc*100

# Prediction on new data

In [ ]:
list=[6.0, 130.0, 72.0, 40.0, 0.0, 25.6, 0.627, 45.0]

In [ ]:
list=torch.tensor(list)

In [ ]:
with torch.no_grad():
    print(model(list).argmax().item())

**It indicates the data is not a diabetes record**